# Finetuning with openinstruct/finetune.py


In [1]:
# job_name = 'ft'
# test_run = 1
# test_run = bool(test_run)

# queue = 'x86_12h' # 'x86_12h'
# num_cpus = 20
# num_gpus = 1
# cpu_mem = 32
# require = 'a100_80gb'

# # model_name_or_path = 'mosaicml/mpt-7b'; max_seq_length = 2048
# # model_name_or_path = 'gpt2'; max_seq_length = 1024
# # model_name_or_path = 'gpt2-Large'; max_seq_length = 1024
# # model_name_or_path = 'gpt2-xl'; max_seq_length = 1024
# model_name_or_path = 'huggyllama/llama-7b'; max_seq_length = 2048


# train_file = 'data/processed/oasst1/oasst1_data.jsonl'; train_file_short = 'oasst1'
# train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; train_file_short = 'human_mix'
# # train_file = 'data/processed/flanv2_cot_oasst1_dolly_shuffled.jsonl'; train_file_short = 'human_mix_shuffled'

# output_dir = f"results/{model_name_or_path.replace('/', ':')}_{train_file_short}"
# if test_run:
#     output_dir = 'jpt_' + output_dir

# use_deepspeed = False
# # deepspeed_config_file = 'ds_configs/stage3_no_offloading_accelerate_setauto.conf'
# # deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate.conf'
# deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate_setauto.conf'

# use_lora = True
# lora_rank = 4
# lora_alpha = lora_rank
# lora_dropout = 0.05

# batch_size_per_gpu = 1
# total_batch_size = 128
# mixed_precision = 'bf16' # 'bf16', 'fp16'
# checkpointing_steps = None # every n steps, where n='1' or every 'epoch'

# gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)

# print(f"Training {model_name_or_path} "
#       f"using {num_gpus} GPUs, "
#       f"{batch_size_per_gpu} batch size per GPU, "
#       f"{gradient_acc_steps} gradient accumulation steps.")

# # do use fast tokenizer since mpt-7b does not have a fast tokenizer counter-part
# #     --use_slow_tokenizer \
# # do not use flash attention, since having problem installing flash-attn with cuda 12.1
# #     --use_flash_attn \

# cmd = f"""
# {'!cd .. && ' if test_run else ''}accelerate launch \
#     --mixed_precision {mixed_precision} \
#     --num_machines 1 \
#     --num_processes {num_gpus} \
#     {'--use_deepspeed' if use_deepspeed else ''}
#     {'--deepspeed_config_file '+deepspeed_config_file if use_deepspeed else ''}
#     open_instruct/finetune.py \
#     --model_name_or_path {model_name_or_path} \
#     --tokenizer_name {model_name_or_path} \
#     --train_file {train_file} \
#     --max_seq_length {max_seq_length} \
#     {'--use_lora' if use_lora else ''}
#     --lora_rank {lora_rank} \
#     --lora_alpha {lora_alpha} \
#     --lora_dropout {lora_dropout} \
#     --preprocessing_num_workers 16 \
#     --per_device_train_batch_size {batch_size_per_gpu} \
#     --gradient_accumulation_steps {gradient_acc_steps} \
#     --learning_rate 2e-5 \
#     --lr_scheduler_type linear \
#     --warmup_ratio 0.03 \
#     --weight_decay 0. \
#     --num_train_epochs 2 \
#     --output_dir {output_dir} \
#     --with_tracking \
#     --report_to tensorboard \
#     {'--checkpointing_steps '+str(checkpointing_steps) if checkpointing_steps else ''}
#     --logging_steps 1
# """

# # things to test to see its effects on (1) eval perf (2) runtime.
# #
# # - int8
# # - mixed_precision bf16 or no
# # - with/without LoRA
# # - LoRA's rank/alpha (alpha typically set to 2*rank)
# # - batch size
# # - micro-batch size (largest without running out of memory)


# cmd = multiline_to_singleline(cmd)
# if test_run:
#     print()
#     print(cmd)


# shell_scripts = shell_scripts_template.format(
#     conda_env='open-instruct',
#     cwd=os.path.dirname(os.getcwd()),
#     cmd=cmd,
#     log_dir=os.getcwd(),
#     save_dir=output_dir
# )
# out = submit_job_ccc(
#     shell_scripts, 
#     job_name=job_name, 
#     queue=queue,
#     num_cpus=num_cpus,
#     cpu_mem=cpu_mem,
#     require=require,
#     num_gpus=num_gpus,
#     test_run=test_run,
# )
# if not test_run:
#     print(out)

# Finetuning with openinstruct/finetune_trainer.py


In [ ]:
def compute_mixture_num_samples(mixture, max_train_samples):
    s = sum(mixture.values())
    mixture = {k: int(max_train_samples*v/s) for k, v in mixture.items()}
    return mixture

add_hardwarespec_to_dirname = False
num_cpus = 144 if arch == 'ppc64le' else 32
cpu_mem =  650 if arch == 'ppc64le' else 64


save_strategy = 'steps'
save_steps = 100
save_total_limit = 1
preprocessing_num_workers = 32
evaluation_strategy = 'no' # set do_eval=False
eval_steps = save_steps
report_to = 'tensorboard wandb'
suffix = None
lr_scheduler_type = 'linear'
warmup_ratio = 0.03


dataloader_sampler = None
hf_models_dir = 'results/baselines/'
# model_name_or_path = 'results/baselines/gpt2-medium'; abbr_model_name = 'gpt2m'; max_seq_length = 1024
model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'; max_seq_length = 2048
# model_name_or_path = 'results/baselines/NousResearch/Llama-2-7b-hf'; abbr_model_name = 'llama2-7b'; max_seq_length = 2048
# model_name_or_path = 'mosaicml/mpt-7b'; abbr_model_name = 'mpt-7b'; max_seq_length = 2048
# model_name_or_path = hf_models_dir+'EleutherAI/pythia-1.4b'; abbr_model_name = 'pythia-1.4b'; max_seq_length = 2048
# model_name_or_path = hf_models_dir+'EleutherAI/pythia-2.8b'; abbr_model_name = 'pythia-2.8b'; max_seq_length = 2048
# model_name_or_path = hf_models_dir+'EleutherAI/pythia-6.9b'; abbr_model_name = 'pythia-6.9b'; max_seq_length = 2048



subsample_mixture_list = []
# subsample_mixture_list += [
#     {k: max_train_samples} for k in datasets
# ]
# subsample_mixture_list += [
#     {k: int(max_train_samples/4) for k in ['cot', 'flan_v2', 'dolly', 'oasst1']}
# ]
# subsample_mixture_list += [
#     ('humanmix', dict((k, int(v*max_train_samples)) for k, v in
#     {'cot': 0.48785105, 'flan_v2': 0.48785105, 'dolly': 0.00732313, 'oasst1': 0.01697478}.items()))
# ] # humanmix mixture.
# subsample_mixture_list += [
#     {k: int(max_train_samples/len(datasets)) for k in datasets} 
# ]
# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {'cot': .07678, 'flan_v2': .9137, 'dolly': .004471, 'oasst1': .009072}.items())
# ]
# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {'cot': 0.1127, 'flan_v2': 0.8726, 'dolly': 0.01395, 'oasst1': 0.001391}.items())
# ]
# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {'cot':  0.13568177819252014, 'flan_v2': 0.3957784175872803, 
#      'dolly': 0.05964866653084755, 'oasst1': 0.4088916480541229}.items())
# ] # gpt2-medium_humanmix_uniform:200k_doremiv2.json

# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {"cot": 0.360595703125, "dolly": 0.0021991729736328125, "flan_v2": 0.63037109375, "oasst1": 0.0016956329345703125}.items())
# ] # pythia-1.4b humanmix_uniform:200k_doremiv1.json

# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {"cot": 0.2254638671875, "dolly": 0.01409149169921875, "flan_v2": 0.1739501953125, "oasst1": 0.59423828125}.items())
# ] # pythia-1.4b humanmix_uniform:200k_doremiv2.json

# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {"cot": 0.08563232421875, "dolly": 0.54296875, "flan_v2": 0.347900390625, "oasst1": 0.0103302001953125}.items())
# ] # llama-7b_humanmix_uniform:200k_doremiv2.json

# subsample_mixture_list += [
#     dict((k, int(v*max_train_samples)) for k, v in
#     {"cot": 0.0316162109375, "dolly": 0.204833984375, "flan_v2": 0.40966796875, "oasst1": 0.40966796875}.items()
#         )] # llama-7b_humanmix_uniform:600k_doremiv2.json

# subsample_mixture_normalized_list = []
# subsample_mixture_normalized_list += [('uniform:1200k_doremiv2', # llama-7b_humanmix_uniform:1200k_doremiv2.json
#                                        {"cot": 0.11419677734375, "dolly": 0.1024169921875, "flan_v2": 0.204833984375, "oasst1": 0.204833984375})]
## 10 for trying out datamodels
# mixes = [{'cot': 0.37664033529374275,
#   'dolly': 0.0874640765523398,
#   'flan_v2': 0.39740799933549775,
#   'oasst1': 0.1384875888184196},
#  {'cot': 0.23064419241874784,
#   'dolly': 0.04693354147889885,
#   'flan_v2': 0.72121745986295,
#   'oasst1': 0.0012048062394032465},
#  {'cot': 0.11244721555034376,
#   'dolly': 0.21997027355988638,
#   'flan_v2': 0.5826671754210359,
#   'oasst1': 0.08491533546873392},
#  {'cot': 0.27704626812045546,
#   'dolly': 0.5712282144637615,
#   'flan_v2': 0.024940119654536592,
#   'oasst1': 0.12678539776124645},
#  {'cot': 0.0024519793352964607,
#   'dolly': 0.13274603201304974,
#   'flan_v2': 0.012268378167304219,
#   'oasst1': 0.8525336104843496},
#  {'cot': 0.08065633865016615,
#   'dolly': 0.41886215168938545,
#   'flan_v2': 0.21723932820070485,
#   'oasst1': 0.2832421814597436},
#  {'cot': 0.13878643021160036,
#   'dolly': 0.05686171157146557,
#   'flan_v2': 0.6701353469446995,
#   'oasst1': 0.13421651127223455},
#  {'cot': 0.2461125374866837,
#   'dolly': 0.09774240280444893,
#   'flan_v2': 0.13974091986040005,
#   'oasst1': 0.5164041398484672},
#  {'cot': 0.4069781049152398,
#   'dolly': 0.06318759506033228,
#   'flan_v2': 0.09504719644992135,
#   'oasst1': 0.4347871035745066},
#  {'cot': 0.22379693013848484,
#   'dolly': 0.30565901275011814,
#   'flan_v2': 0.15457716965000887,
#   'oasst1': 0.31596688746138824}]

# mixes = [
#     {'cot': 0.46638974, 'dolly': 0.01456044, 'flan_v2': 0.50886009, 'oasst1': 0.01018973},
#     {'cot': 0.39744481, 'dolly': 0.00472114, 'flan_v2': 0.59104177, 'oasst1': 0.00679229},
# ]

# subsample_mixture_normalized_list += [('', d) for d in mixes]
# subsample_mixture_normalized_list += [('humanmix', # humanmix
#                                        {'cot': 0.48785105, 'dolly': 0.00732313, 'flan_v2': 0.48785105, 'oasst1': 0.01697478})]
# subsample_mixture_normalized_list = [(x[0],  compute_mixture_num_samples(x[1], max_train_samples)) 
#                                      for x in subsample_mixture_normalized_list]
# subsample_mixture_list += subsample_mixture_normalized_list


subsample_mixture_list = [('',None)]
subsample_inds_file_list = [None]


train_file = 'data/processed/all.jsonl'; abbr_train_file = 'all'



max_train_samples_list = [None]
num_train_epochs_list = [1]




def subsample_inds_file_abbr_fn(x):
    s = os.path.basename(x).split('.pkl')[0]
    if s.startswith('inds_'):
        scoring_fn = os.path.basename(os.path.dirname(x)).replace('_', ':')
        pacing_fn = s.split('inds_')[-1].replace('_', ':')
        return f'score={scoring_fn}_pace={pacing_fn}'
    else:
        return s

##  ft1: reproduce open-instruct table with llama7b
# job_name = 'ft1'; num_train_epochs_list = [2]
# job_name = 'ft1_ep=1'; num_train_epochs_list = [1] # train for 1 epoch (baseline for comparison.)
# job_name = 'ft1_ep=2'; num_train_epochs_list = [2]
# job_name = 'oi2'; num_train_epochs_list = [2] # 5, 10
# model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'; max_seq_length = 2048

# train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm'

# train_file = 'data/processed/lima/lima_data.jsonl'; abbr_train_file = 'lima'
# train_file = 'data/processed/cot/cot_data.jsonl'; abbr_train_file = 'cot'
# train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2'
# # # train_file = 'data/processed/wpq/cot_flanv2_data.jsonl'; abbr_train_file = 'cot:flanv2'
# # # train_file = 'data/processed/tulu/tulu_v1_human_mix.jsonl'; abbr_train_file = 'hmv1'
# train_file = 'data/processed/tulu/tulu_v1_mix.jsonl'; abbr_train_file = 'tuluv1m'
# train_file = 'data/processed/sharegpt/sharegpt_data.jsonl'; abbr_train_file = 'sharegpt'



# model_name_or_path = hf_models_dir+'mistralai/Mistral-7B-v0.1'; abbr_model_name = 'mistral-7b'; max_seq_length = 2048
# train_file = 'data/processed/ultrachat/ultrachat200k_train_data.jsonl'; abbr_train_file = 'ultrachat200k'
# train_file = 'data/processed/lima/lima_data.jsonl'; abbr_train_file = 'lima'

# # max_train_samples_list = [120]; save_steps = 1; save_total_limit = 100



# # train_file = 'data/processed/dolly/dolly_data.jsonl'; abbr_train_file = 'dolly'
# # train_file = 'data/processed/oasst1/oasst1_data.jsonl'; abbr_train_file = 'oasst1'
# # train_file = 'data/processed/super_ni/super_ni_data.jsonl'; abbr_train_file = 'super_ni'
# # train_file = 'data/processed/stanford_alpaca/stanford_alpaca_data.jsonl'; abbr_train_file = 'stanford_alpaca'
# # train_file = 'data/processed/baize/baize_data.jsonl'; abbr_train_file = 'baize'
# # train_file = 'data/processed/self_instruct/self_instruct_data.jsonl'; abbr_train_file = 'self_instruct'
# # train_file = 'data/processed/code_alpaca/code_alpaca_data.jsonl'; abbr_train_file = 'code_alpaca'
# # train_file = 'data/processed/unnatural_instructions/unnatural_instructions_data.jsonl'; abbr_train_file = 'unnatural_instructions'
# # train_file = 'data/processed/gpt4_alpaca/gpt4_alpaca_data.jsonl'; abbr_train_file = 'gpt4_alpaca'


# # ft2: test mixture weights
# # vary mixture weights
# job_name = 'ft2'

# # oi3: instruction tuning performance w.r.t. steps.
# job_name = 'oi3'

# # oi4: data pruning 
# job_name = 'oi4_flan_v2_vary_subsetsize'
# train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2'
# subsample_mixture_list = [('',None)]
# max_train_samples_list = [int(pct*100000) for pct in [.1, .3, .5]]; num_train_epochs_list = [2]
# data_inds_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/llama-7b/flan_v2/'
# subsample_inds_file_list = [
# #     os.path.join(data_inds_dir, 'random.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'prob_decr.pkl'),
# #     os.path.join(data_inds_dir, 'prob_incr.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_k=Kcos.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_k=Kcosp.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_k=Kcos1np.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_incr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_decr.pkl'),
#     os.path.join(data_inds_dir, 'kmeanscd_nc=3000_incr.pkl'),
#     os.path.join(data_inds_dir, 'kmeanscd_nc=3000_decr.pkl'),
# ]


# # oi4_perf_cross_time: perf cross time on flan_v2
# job_name = 'oi4_perf_cross_time'
# save_steps = 50; save_total_limit = 200
# train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2'
# subsample_mixture_list = [('',None)]
# max_train_samples_list = [int(pct*100000) for pct in [.3]]; num_train_epochs_list = [3]
# data_inds_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/llama-7b/flan_v2/'
# subsample_inds_file_list = [
#     os.path.join(data_inds_dir, 'random.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=300_decr.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=300_incr.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=1000_decr.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=1000_incr.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_decr.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_incr.pkl'),
#     os.path.join(data_inds_dir, 'prob_decr.pkl'),
#     os.path.join(data_inds_dir, 'prob_incr.pkl'),
#     os.path.join(data_inds_dir, 'dppmap_k=Kcos.pkl'),
#     os.path.join(data_inds_dir, 'dppmap_k=Kcosp.pkl'),
#     os.path.join(data_inds_dir, 'dppmap_k=Kcos1np.pkl'),
#     os.path.join(data_inds_dir, 'el2n_incr.pkl'),
#     os.path.join(data_inds_dir, 'el2n_decr.pkl'),
# ]

# # ## oi4_flanv2_prune_with_hmv1_model
# job_name = 'oi4_flanv2_prune_with_hmv1_model'
# save_steps = 50; save_total_limit = 200
# train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2'
# subsample_mixture_list = [('',None)]
# max_train_samples_list = [int(pct*100000) for pct in [.3]]; num_train_epochs_list = [3]
# data_inds_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/llama-7b_ft=hmv1/flan_v2/'
# subsample_inds_file_list = [
# #     os.path.join(data_inds_dir, 'random.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_incr.pkl'),
#     os.path.join(data_inds_dir, 'prob_incr.pkl'),
# #     os.path.join(data_inds_dir, 'prob_decr.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_k=Kcos.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_k=Kcosp.pkl'),
#     os.path.join(data_inds_dir, 'el2n_incr.pkl'),
#     os.path.join(data_inds_dir, 'el2n_decr.pkl'),
# ]

# ## tulu mix v1.
# dataset = 'tulu_v1_human_mix'; train_file = 'data/processed/tulu/tulu_v1_human_mix.jsonl'; abbr_train_file = 'tuluv1hm'
# job_name = f'oi4_{dataset}'
# save_steps = 50; save_total_limit = 200
# subsample_mixture_list = [('',None)]
# max_train_samples_list = [30000]; num_train_epochs_list = [3]
# data_inds_dir = f'/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/llama-7b/{dataset}/'
# subsample_inds_file_list = [
#     os.path.join(data_inds_dir, 'random.pkl'),
#     os.path.join(data_inds_dir, 'prob_decr.pkl'),
#     os.path.join(data_inds_dir, 'prob_incr.pkl'),
#     os.path.join(data_inds_dir, 'dppmap_k=Kcos.pkl'),
#     os.path.join(data_inds_dir, 'dppmap_k=Kcosp.pkl'),
#     os.path.join(data_inds_dir, 'dppmap_k=Kcos1np.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_decr.pkl'),
#     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_incr.pkl'),
# ]


# ## 
# dataset = 'flan2022_1m'; train_file = 'data/processed/flan2022/flan2022_1m_data.jsonl'; abbr_train_file = 'flan2022_1m'
# job_name = f'oi4_{dataset}'
# save_steps = 50; save_total_limit = 200
# subsample_mixture_list = [('',None)]
# data_inds_dir = f'/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/llama-7b+lora:r=256:a=256/{dataset}/'
# ## full data
# # max_train_samples_list = [1000000]; num_train_epochs_list = [1]
# # subsample_inds_file_list = ['']
# # subset
# max_train_samples_list = [200000]; num_train_epochs_list = [1]
# subsample_inds_file_list = [
# #     os.path.join(data_inds_dir, 'random.pkl'),
# #     os.path.join(data_inds_dir, 'prob_decr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_nc=3000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeanscd_nc=3000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeanscd_nc=3000_decr.pkl'),
#     # not that helpful
# #     os.path.join(data_inds_dir, 'prob_incr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_decr.pkl'),
#     ## gradnorm outputs
# #     os.path.join(data_inds_dir, 'el2n_agg=mean_incr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_agg=l2n_incr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_agg=l2n_decr.pkl'),
# #     os.path.join(data_inds_dir, 'logit_margin_incr.pkl'),
# #     os.path.join(data_inds_dir, 'logit_margin_decr.pkl'),
# #     os.path.join(data_inds_dir, 'grad_loraB_l2n_incr.pkl'),
# #     os.path.join(data_inds_dir, 'grad_loraB_l2n_decr.pkl'),  
#     ## random baselines.
# #     os.path.join(data_inds_dir, 'random_s=0.pkl'),
# #     os.path.join(data_inds_dir, 'random_s=1.pkl'),
# #     os.path.join(data_inds_dir, 'random_s=2.pkl'),
#     ## kmeans on grads
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=1000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=1000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=3000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=3000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=6000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=6000_incr.pkl'),
# ]



# ## 
# dataset = 'tulu_v1_mix'; train_file = 'data/processed/tulu/tulu_v1_mix.jsonl'; abbr_train_file = 'tuluv1m'
# # job_name = f'oi4_{dataset}_ep=3'
# job_name = f'oi5_tulu_v1_mix:llama-7b' # re-run to see if transformers upgrade altered eval performance.
# # save_steps = 50; save_total_limit = 200
# subsample_mixture_list = [('',None)]
# data_inds_dir = f'/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/data_inds/llama-7b+lora:r=256:a=256/{dataset}/'
# max_train_samples_list = [50000]; num_train_epochs_list = [3]
# subsample_inds_file_list = [
#     # random baselines
# #     os.path.join(data_inds_dir, 'random_s=0.pkl'),
# #     os.path.join(data_inds_dir, 'random_s=1.pkl'),
# #     os.path.join(data_inds_dir, 'random_s=2.pkl'),
# #     # correlated statistics
# #     os.path.join(data_inds_dir, 'log_prob_incr.pkl'),
#     os.path.join(data_inds_dir, 'log_prob_decr.pkl'),
# #     os.path.join(data_inds_dir, 'logit_margin_incr.pkl'),
# #     os.path.join(data_inds_dir, 'logit_margin_decr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_agg=mean_incr.pkl'),
# #     os.path.join(data_inds_dir, 'el2n_agg=mean_decr.pkl'),
# #     # grad norm
# #     os.path.join(data_inds_dir, 'grad_loraB_l2n_incr.pkl'),
# #     os.path.join(data_inds_dir, 'grad_loraB_l2n_decr.pkl'),  
# #     # kmeans
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=1000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=1000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=text+embedding_nc=1000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=text+embedding_nc=1000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=3000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=grad+rp+loraB_nc=3000_decr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=text+embedding_nc=3000_incr.pkl'),
# #     os.path.join(data_inds_dir, 'kmeansl2_emb=text+embedding_nc=3000_decr.pkl'),
# # #     # kcos only 50k data
# #     os.path.join(data_inds_dir, 'dppmap_emb=grad+rp+loraB_k=Kcos.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_emb=text+embedding_k=Kcos.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_emb=grad+rp+loraB_k=Kcos1np.pkl'),
# #     os.path.join(data_inds_dir, 'dppmap_emb=text+embedding_k=Kcos1np.pkl'),
# ]

############
# oi5: try curriculum learning / pruning

scoring_fn_and_pacing_fn = []

# ###### 150k(ep=3) baselines on various baselines
# model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'; max_seq_length = 2048
# M = 150_000; dataset = 'tulu_v1_mix'; train_file = 'data/processed/tulu/tulu_v1_mix.jsonl'; abbr_train_file = 'tuluv1m'; pacing_fn_list = [f'prune_size={M}_ep=3']
# # M = 50_000; dataset = 'tulu_v1_mix'; train_file = 'data/processed/tulu/tulu_v1_mix.jsonl'; abbr_train_file = 'tuluv1m'; pacing_fn_list = [f'prune_size={M}_ep=5'] 


# # M = 150_000; dataset = 'wizardlm'; train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm'
# # M = 150_000; dataset = 'sharegpt'; train_file = 'data/processed/sharegpt/sharegpt_data.jsonl'; abbr_train_file = 'sharegpt'
# # M = 150_000; dataset = 'ultrachat200k'; train_file = 'data/processed/ultrachat/ultrachat200k_train_data.jsonl'; abbr_train_file = 'ultrachat200k'

# scoring_fn_list = []
# # scoring_fn_list += ['random_s=0', 'random_s=1']
# # scoring_fn_list += ['dppmap_k=vmf_gamma=0.0315_kmd=mpnet']
# # scoring_fn_list += ['dppmap_k=rbf_gamma=auto10000_kmd=llama7b_kemb=text+embedding']
# scoring_fn_list = ['log_pmi_neg', 'ifd_neg']
# # pacing_fn_list = [
# # #     f'prune_size={M}_ep=3',
# #     f'singlestep_size={M}_startingfrac=0.05',
# # #     f'singlestep_size={M}_startingfrac=0.1',
# # #     f'singlestep_size={M}_startingfrac=0.2',
# #     f'fep_size={M}_nsteps=5_startingfrac=0.05_inc=1.5',
# # ]
# scoring_fn_and_pacing_fn += list(itertools.product(scoring_fn_list, pacing_fn_list))
# ######

# ##### codellama on starcoder. 5k subset ep=5
# model_name_or_path = hf_models_dir+'codellama/CodeLlama-7b-hf'; abbr_model_name = 'codellama-7b'; max_seq_length = 2048
# # M = 25_000; dataset='starcoder_commentinstr'; train_file = 'data/processed/starcoder/starcoder_commentinstr.jsonl'; abbr_train_file = 'starcodercmtinstr'; pacing_fn_list = [f'prune_size={M}_ep=5']; subset_size = 5_000
# # M = 25_000; dataset='starcoder_commentinstr_cleaned'; train_file = 'data/processed/starcoder/starcoder_commentinstr_cleaned.jsonl'; abbr_train_file = 'starcodercmtinstrcleaned'; pacing_fn_list = [f'prune_size={M}_ep=5'] ; subset_size = 5_000
# # M = 50_000; dataset='starcoder_commentinstrv2'; train_file = 'data/processed/starcoder/starcoder_commentinstrv2.jsonl'; abbr_train_file = 'starcodercmtinstrv2'; pacing_fn_list = [f'prune_size={M}_ep=5'] ; subset_size = 10_000
# M = 50_000; dataset='starcoder_commentinstrv5'; train_file = 'data/processed/starcoder/starcoder_commentinstrv5.jsonl'; abbr_train_file = 'starcodercmtinstrv5'; pacing_fn_list = [f'prune_size={M}_ep=5'] ; subset_size = 10_000
# # M = 60_000; dataset='starcoder_commentinstrv5'; train_file = 'data/processed/starcoder/starcoder_commentinstrv5.jsonl'; abbr_train_file = 'starcodercmtinstrv5'; pacing_fn_list = [f'prune_size={M}_ep=3'] ; subset_size = 20_000

# scoring_fn_list = []
# scoring_fn_list += ['log_prob_neg', 'el2n_agg=mean', 'grad_loraB_l2n', 'ifd_neg', 'log_pmi_neg']

# # scoring_fn_list += ['log_pmi_neg']
# # scoring_fn_list += ['random_s=0', 'random_s=1']
# # scoring_fn_list += ['dedup_md=mpnet_emb=text+embedding']

# # scoring_fn_list += [
# #     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=mpnet',
# #     f'dppmap_k=rbf_gamma=auto{subset_size}_kmd=llama7b_kemb=text+embedding',
# #     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=codellama7b_kemb=grad+rp+loraB',
# # ]

# scoring_fn_and_pacing_fn += list(itertools.product(scoring_fn_list, pacing_fn_list))
# #####

# ##### 
# model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'; max_seq_length = 2048
# # M = 150_000; dataset = 'tulu_v1_mix'; train_file = 'data/processed/tulu/tulu_v1_mix.jsonl'; abbr_train_file = 'tuluv1m'; pacing_fn_list = [f'prune_size={M}_ep=3']
# # M = 200_000; dataset = 'wizardlm'; train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm'; pacing_fn_list = [f'prune_size={M}_ep=2']
# # M = 100_000; dataset = 'wizardlm'; train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm'; pacing_fn_list = [f'prune_size={M}_ep=2'] # 6hr gpu ok.
# M = 50_000; dataset = 'wizardlm'; train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm'; pacing_fn_list = [f'prune_size={M}_ep=5'] # 6hr gpu ok.
# # M = 10_000; dataset = 'wizardlm'; train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm'; pacing_fn_list = [f'prune_size={M}_ep=10'] # 6hr gpu ok.


# scoring_fn_list = []
# # scoring_fn_list += ['log_prob_neg', 'el2n_agg=mean', 'grad_loraB_l2n'] # 'numtoks_input_neg'
# # scoring_fn_list += ['ifd', 'ifd_neg', 'log_pmi', 'log_pmi_neg']
# # scoring_fn_list += ['dedup_md=mpnet_emb=text+embedding']
# # scoring_fn_list += ['semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=200',
# # #                     'semdedup_cl=kmeansfaisscd_md=bge_dist=cd_emb=text+embedding_nc=200'
# #                    ]
# # scoring_fn_list = ['semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=200',
# #                    'semdedup_cl=kmeansfaisscd_md=bge_dist=cd_emb=text+embedding_nc=200',
# #                    'semdedup_cl=kmeansfaisscd_md=llama7b_dist=cd_emb=text+embedding_nc=200',
# #                    'semdedup_cl=kmeansfaisscd_md=llama7b_dist=cd_emb=grad+rp+loraB_nc=200',]
# # scoring_fn_list += ['random_s=0', 'random_s=1']
# scoring_fn_list = [ 
#     # ->1k
# #     'dppmap:k=lin:kmd=mpnet',
# #     'dppmap_k=vmf_gamma=auto1000_kmd=bge'
# #     'dppmap_k=vmf_gamma=0.008_kmd=mpnet',
# #     'dppmap_theta=0.5_k=vmf_gamma=0.01_kmd=mpnet_q=log+pmi_qmd=llama7b',
# #     'dppmap_theta=0.3_k=vmf_gamma=0.01_kmd=mpnet_q=log+pmi_qmd=llama7b',
# #     'dppmap_k=rbf_gamma=0.0000007_kmd=llama7b_kemb=grad+rp+loraB',
# #     'dppmap_k=rbf_gamma=3.5e-8_kmd=llama7b_kemb=text+embedding',
# #     'dppmap_k=vmf_gamma=auto1000_kmd=llama7b_kemb=text+embedding',
# #     'dppmap_k=vmf_gamma=auto1000_kmd=llama7b_kemb=grad+rp+loraB',
#     # ->10k
# #     'dppmap_k=vmf_gamma=auto10000_kmd=bge'
# #     'dppmap_theta=0.5_k=vmf_gamma=0.043_kmd=mpnet_q=log+pmi_qmd=llama7b',
# #     'dppmap_theta=0.7_k=vmf_gamma=0.043_kmd=mpnet_q=log+pmi_qmd=llama7b',
# #     'dppmap_theta=0.3_k=vmf_gamma=0.04_kmd=mpnet_q=ifd_qmd=llama7b',
# #     'dppmap_theta=0.3_k=vmf_gamma=0.043_kmd=mpnet_q=ifd+neg_qmd=llama7b',
# #     'dppmap_k=rbf_gamma=7.5e-06_kmd=llama7b_kemb=text+embedding',
# #     'dppmap_k=rbf_gamma=0.001_kmd=llama7b_kemb=grad+rp+loraB',
# #     'dppmap_k=vmf_gamma=auto10000_kmd=llama7b_kemb=text+embedding',
# #     'dppmap_k=vmf_gamma=auto10000_kmd=llama7b_kemb=grad+rp+loraB',
#     'dppmap_theta=0.1_k=rbf_gamma=auto10000_kmd=llama7b_kemb=text+embedding_q=log+pmi_qmd=llama7b',
#     'dppmap_theta=0.05_k=rbf_gamma=auto10000_kmd=llama7b_kemb=text+embedding_q=log+pmi_qmd=llama7b',
# #     'dppmap_theta=0.3_k=rbf_gamma=auto10000_kmd=llama7b_kemb=text+embedding_q=log+pmi_qmd=llama7b',
# #     'dppmap_theta=0.5_k=rbf_gamma=auto10000_kmd=llama7b_kemb=text+embedding_q=log+pmi_qmd=llama7b',
# ]
# scoring_fn_and_pacing_fn += list(itertools.product(scoring_fn_list, pacing_fn_list))
# ####

##### sharegptv2, open_orca_slim
# model_name_or_path = hf_models_dir+'NousResearch/Llama-2-7b-hf'; abbr_model_name = 'llama2-7b'; max_seq_length = 2048 # 4096
# model_name_or_path = hf_models_dir+'NousResearch/Llama-2-13b-hf'; abbr_model_name = 'llama2-7b'; max_seq_length = 2048 # 4096
model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'; max_seq_length = 2048

# dataset = 'wizardlm'; train_file = 'data/processed/wizardlm/wizardlm_data.jsonl'; abbr_train_file = 'wizardlm';
# dataset = 'wizardlmv2'; train_file = 'data/processed/wizardlm/wizardlmv2_data.jsonl'; abbr_train_file = 'wizardlmv2';
# dataset = 'dolly'; train_file = 'data/processed/dolly/dolly_data.jsonl'; abbr_train_file = 'dolly';
dataset = 'flan_v2'; train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2';
# dataset = 'oasst1'; train_file = 'data/processed/oasst1/oasst1_data.jsonl'; abbr_train_file = 'oasst1'; 
# dataset = 'sharegptv2'; train_file = 'data/processed/sharegpt/sharegptv2_data.jsonl'; abbr_train_file = 'sharegptv2';
# dataset = 'stanford_alpaca'; train_file = 'data/processed/stanford_alpaca/stanford_alpaca_data.jsonl'; abbr_train_file = 'stanford_alpaca';
# dataset = 'ultrachat200kv2'; train_file = 'data/processed/ultrachat/ultrachat200kv2_train_data.jsonl'; abbr_train_file = 'ultrachat200kv2';
# dataset = 'open_orca_slim'; train_file = 'data/processed/open_orca/open_orca_slim_data.jsonl'; abbr_train_file = 'openorcaslim'; 
# dataset = 'tulu_v2'; train_file = 'data/processed/tulu_v2/tulu_v2_data.jsonl'; abbr_train_file = 'tulu_v2';

# M = 60_000; pacing_fn_list = [f'prune_size={M}_ep=3']; subset_size = 20_000
# M = 50_000; pacing_fn_list = [f'prune_size={M}_ep=5']; subset_size = 10_000
M = 20_000; pacing_fn_list = [f'prune_size={M}_ep=4']; subset_size = 5_000
# M = 10_000; pacing_fn_list = [f'prune_size={M}_ep=10']; subset_size = 1_000

scoring_fn_list = []

scoring_fn_list += ['random_s=0'] # , 'random_s=1'
# scoring_fn_list += ['random_s=1']
# scoring_fn_list += ['log_prob_neg', 'el2n_agg=mean', 'grad_loraB_l2n', 'ifd_neg', 'log_pmi_neg']

# scoring_fn_list += [
#     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=mpnet',
#     f'dppmap_k=rbf_gamma=auto{subset_size}_kmd=llama7b_kemb=text+embedding',
#     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=llama7b_kemb=grad+rp+loraB',
# #     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=llama7b_kemb=text+embedding',
# #     f'dppmap_k=rbf_gamma=auto{subset_size}_kmd=llama7b_kemb=grad+rp+loraB',
# ]
# subset_size = 10_000
scoring_fn_list += [ # autotune gamma
#     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=mpnet',
#     f'dppmap_k=rbf_gamma=auto{subset_size}_kmd=llama7br512p4096_kemb=text+embedding',
#     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd=llama7br512p4096_kemb=grad+rp+loraB',
]
scoring_fn_list += [ # vary kernel embedding model 
#     f'dppmap_k=vmf_gamma=auto{subset_size}_kmd={kmd}_kemb=grad+rp+loraB'
#     for kmd in ['llama7br256p4096', 'llama7br512p4096', 'pythia1br512p4096']
]
scoring_fn_list += [ # use a sufficiently large gamma
    f'dppmap_k=vmf_gamma=1_kmd=llama7br512p4096_kemb=grad+rp+loraB',
#     f'dppmap_k=vmf_gamma=1e-3_kmd=llama7br512p4096_kemb=grad+rp+loraB',
]
scoring_fn_and_pacing_fn += list(itertools.product(scoring_fn_list, pacing_fn_list))
##### 



# ### mistral-7b on ultrachat15/200k
# model_name_or_path = hf_models_dir+'mistralai/Mistral-7B-v0.1'; abbr_model_name = 'mistral-7b'; max_seq_length = 2048
# # M =  50_000; dataset = 'ultrachat200k'; train_file = 'data/processed/ultrachat/ultrachat200k_train_data.jsonl'; abbr_train_file = 'ultrachat200k'
# M = 100_000; dataset = 'ultrachat15';   train_file = 'data/processed/ultrachat/ultrachat15_data.jsonl'; abbr_train_file = 'ultrachat15'; preprocessing_num_workers = 64
# # M = 400_000; dataset = 'ultrachat15';   train_file = 'data/processed/ultrachat/ultrachat15_data.jsonl'; abbr_train_file = 'ultrachat15'; preprocessing_num_workers = 64

# # scoring_fn_list = ['log_prob_neg', 'el2n_agg=mean', 'grad_loraB_l2n', 'el2n_agg=mean']
# # scoring_fn_list = ['numtoks_input_neg']
# # scoring_fn_list = ['log_prob_neg', 'el2n_agg=mean', 'logit_margin_neg', 'grad_loraB_l2n',] #  'kmeansl2_emb=text+embedding_nc=3000_incr'
# # scoring_fn_list = ['rhov1_log_prob', 'rhov1_log_prob_neg']
# # scoring_fn_list = ['numtoks_input_neg', 'numtoks_output_neg', 'numtoks_total_neg']
# # scoring_fn_list = [
# #     'semdedup_cl=kmeansfaisscd_md=mpnet_dist=cd_emb=text+embedding_nc=200',
# #                    'semdedup_cl=kmeansfaisscd_md=mistral7b_dist=cd_emb=text+embedding_nc=200',
# #                    'semdedup_cl=kmeansfaisscd_md=mistral7b_dist=cd_emb=grad+rp+loraB_nc=200',
# #                   ]
# # scoring_fn_list = ['dppmapbd_nc=200_k=lin_kmd=mpnet']
# # scoring_fn_list =[f'dppmapbd_nc=200_k=vmf_gamma={gamma}_kmd=mpnet' for gamma in [.3, 3.]]
# # scoring_fn_list = ['dppmapbd_nc=200_k=vmf_gamma=0.000035_kmd=mpnet']
# scoring_fn_list = ['dppmapbd_nc=200_k=vmf_gamma=1.0_kmd=mpnet'] 
# pacing_fn_list = [f'prune_size={M}_ep=2']
# scoring_fn_and_pacing_fn += list(itertools.product(scoring_fn_list, pacing_fn_list))
# ##### 

evaluation_strategy = 'steps' if 'ultrachat' in dataset else 'no'
job_name = f'oi5_{dataset}:{abbr_model_name}'
num_train_epochs_list = [1] # offload handling of epochs to `generate_curriculum`
dataloader_sampler = 'SequentialSampler'

## random baselines # 'random_s=0', 'random_s=1'
# scoring_fn_list = ['random_s=0']; pacing_fn_list = [f'prune_size={M}_ep=2']
# scoring_fn_and_pacing_fn += list(itertools.product(scoring_fn_list, pacing_fn_list))

subsample_inds_file_list = []
for scoring_fn, pacing_fn in scoring_fn_and_pacing_fn:
    from note_pruning import get_final_model_name
    from note_pruning_analysis import get_full_model_name
    md_expand = get_final_model_name(abbr_model_name, scoring_fn)
    if dataset in ['wizardlmv2', 'ultrachat200kv2']: # only relied on llama7br512p4096
        if 'llama' in abbr_model_name:
            if not any(y in scoring_fn for y in ['mpnet', 'bge']):
                md_expand = get_full_model_name('llama7br512p4096')
        else:
            raise ValueError('need to specify md_expand!')
    if md_expand == 'llama-7b': md_expand = get_full_model_name('llama7b')
    print(f'md={abbr_model_name} scoring_fn={scoring_fn}\t->\tmd_expand={md_expand}')
    data_inds_dir = (f'/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/curriculum/'
                     f'{md_expand}/{dataset}/')
    p = os.path.join(data_inds_dir, scoring_fn, 'inds_'+pacing_fn+'.pkl')
    if not os.path.isfile(p):
        raise ValueError(f'path={p} does not exists for {scoring_fn}')
    subsample_inds_file_list.append(p)

#####



# #### sharegptv2 & open_orca_slim baselines
# job_name = 'oi2'; num_train_epochs_list = [2] # 5, 10
# # model_name_or_path = hf_models_dir+'NousResearch/Llama-2-7b-hf'; abbr_model_name = 'llama2-7b'; max_seq_length = 2048
# model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; abbr_model_name = 'llama-7b'; max_seq_length = 2048
# # train_file = 'data/processed/sharegpt/sharegptv2_data.jsonl'; abbr_train_file = 'sharegptv2'
# # train_file = 'data/processed/open_orca/open_orca_slim_data.jsonl'; abbr_train_file = 'openorcaslim'; max_train_samples_list=[100_000]
# # train_file = 'data/processed/stanford_alpaca/stanford_alpaca_data.jsonl'; abbr_train_file = 'stanford_alpaca'; 
# # train_file = 'data/processed/flan_v2/flan_v2_data.jsonl'; abbr_train_file = 'flan_v2';
# # train_file = 'data/processed/oasst1/oasst1_data.jsonl'; abbr_train_file = 'oasst1';
# # train_file = 'data/processed/dolly/dolly_data.jsonl'; abbr_train_file = 'dolly';
# train_file = 'data/processed/wizardlm/wizardlmv2_data.jsonl'; abbr_train_file = 'wizardlmv2';

# # train_file = 'data/processed/tulu_v2/tulu_v2_data.jsonl'; abbr_train_file = 'tulu_v2'; max_train_samples_list=[100_000]
# ####


# ##### code instruction tuning
# model_name_or_path = hf_models_dir+'codellama/CodeLlama-7b-hf'; abbr_model_name = 'codellama-7b'; max_seq_length = 2048

# # job_name = 'oi6_starcoder_ep=5'; # 5k train for 5 epochs
# # num_train_epochs_list = [5]
# # train_file = 'data/processed/starcoder/starcoder_commentinstr.jsonl'; abbr_train_file = 'starcodercmtinstr'
# # train_file = 'data/processed/starcoder/starcoder_commentinstr_cleaned.jsonl'; abbr_train_file = 'starcodercmtinstrcleaned'
# # train_file = 'data/processed/starcoder/starcoder_commentinstrv2.jsonl'; abbr_train_file = 'starcodercmtinstrv2'; max_train_samples_list=[5000]
# # train_file = 'data/processed/starcoder/starcoder_commentinstrv2_flppl.jsonl'; abbr_train_file = 'starcodercmtinstrv2_flppl'; max_train_samples_list=[5000]
# # train_file = 'data/processed/starcoder/starcoder_commentinstrv4_rge5.jsonl'; abbr_train_file = 'starcodercmtinstrv4_rge5'; max_train_samples_list=[5000]
# # train_file = 'data/processed/starcoder/starcoder_commentinstrv4.jsonl'; abbr_train_file = 'starcodercmtinstrv4'; max_train_samples_list=[5000]

# job_name = 'oi2'; 
# num_train_epochs_list = [2]
# # train_file = 'data/processed/starcoder/starcoder_commentinstrv2.jsonl'; abbr_train_file = 'starcodercmtinstrv2'
# train_file = 'data/processed/starcoder/starcoder_commentinstrv5.jsonl'; abbr_train_file = 'starcodercmtinstrv5'

# ##### 





test_run = 1
test_run = bool(test_run)
debug_mode = test_run

nodes = 5; num_gpus = 6; gpu_type = 'v100'; job_duration = 6 # llama-7b on 100k. data
# nodes = 5; num_gpus = 6; gpu_type = 'v100'; job_duration = 12 # llama-7b on 100k. data
# nodes = 10; num_gpus = 6; gpu_type = 'v100'; job_duration = 6 # llama-7b on 100k. data
# nodes = 5; num_gpus = 6; gpu_type = 'v100'; job_duration = 18 # llama-7b on 100k. data

# nodes = 5; num_gpus = 6; gpu_type = 'v100'; job_duration = 18 # llama-7b on 400k data
# nodes = 5; num_gpus = 6; gpu_type = 'v100'; job_duration = 30 # llama-7b on 600k data

# nodes = 1; num_gpus = 1; gpu_type = 'v100'; job_duration = 6  # gpt2
# nodes = 2; num_gpus = 6; gpu_type = 'v100'; job_duration = 6  # pythia-1.4b
# nodes = 5; num_gpus = 6; gpu_type = 'v100'; job_duration = 6  # pythia-2.8b|6.9b


overwrite_output_dir = True if test_run else False # always continue from ckpt if run from cluster.


per_device_train_batch_size = 2; total_batch_size = 128 # 128
gradient_accumulation_steps = round(total_batch_size/(num_gpus*nodes)/per_device_train_batch_size)
effective_batch_size = per_device_train_batch_size*nodes*num_gpus*gradient_accumulation_steps


optimizer = 'adamw_hf' # 'adafactor'

deepspeed = ''; fsdp = False if num_gpus == 1 else "full_shard auto_wrap"  # full_shard, shard_grad_op
if 'gpt2' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'GPT2Block'
elif 'llama' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'LlamaDecoderLayer'
elif 'mpt' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'MPTBlock'
elif 'pythia' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'GPTNeoXLayer'        
elif 'mistral' in abbr_model_name: fsdp_transformer_layer_cls_to_wrap = 'MistralDecoderLayer'
else: raise ValueError('Not sure how to set `fsdp_transformer_layer_cls_to_wrap`')
    

# deepspeed = './ds_configs/ds_zero3_cpu_offload.json'; fsdp = False
# deepspeed = './ds_configs/ds_zero3.json'; fsdp = False
# deepspeed = './ds_configs/stage3_no_offloading.conf'; fsdp = False # error with loading... something wrong with the config.

# fsdp = False; deepspeed = False

if fsdp and deepspeed:
    raise ValueError('either fsdp or deepspeed, not both')

use_lora = False
lora_rank = 256 # test {8, 16, 32, 128} # just [128, 8] for now.
lora_alpha = lora_rank 
lora_dropout = 0.05
if use_lora:
    abbr_model_name += f'+lora(r={lora_rank},a={lora_alpha})'

mixed_precision = 'bf16' if arch == 'x86_64' else 'fp16' # ; mixed_precision = None
torch_dtype = 'bfloat16' if arch=='x86_64' else 'float16'; torch_dtype = 'float32'

gradient_checkpointing = True
load_in_8bit = False



print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{per_device_train_batch_size} batch size per GPU, "
      f"{gradient_accumulation_steps} gradient accumulation steps, "
      f"Effective batch size {effective_batch_size}")


if nodes == 1:
    exe = 'python' if num_gpus==1 else \
        f"torchrun --nproc_per_node={num_gpus} --master_port=10002"
else:
    exe = f"torchrun --nnodes={nodes} --nproc_per_node={num_gpus} --rdzv-id=$SLURM_JOB_ID --rdzv-backend=c10d --rdzv-endpoint=$RDZV_ENDPOINT"

if test_run:
    exe = f"CUDA_VISIBLE_DEVICES={','.join(map(str, range(num_gpus)))} {exe}"
if test_run and debug_mode:
    exe = 'TORCH_CPP_LOG_LEVEL=INFO NCCL_DEBUG=INFO LOGLEVEL=INFO ' + exe
    error_file='/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/external/open-instruct/scripts/error_file'
    exe = f'TORCHELASTIC_ERROR_FILE={error_file} {exe}'

if not os.path.isfile(train_file):
    print(f'train_file={train_file} does not exists')

options_list = itertools.product(
    num_train_epochs_list,
    subsample_mixture_list,
    subsample_inds_file_list,
    max_train_samples_list,
)

output_dirname_list = []
for (num_train_epochs,
     mix_name_and_subsample_mixture,
     subsample_inds_file,
     max_train_samples,) in options_list:
    mix_name, subsample_mixture = mix_name_and_subsample_mixture

    output_dirname = f"{abbr_model_name}_{abbr_train_file}"
    if max_train_samples:
        output_dirname += f":{int(max_train_samples/1000)}k"
        
    if job_name == 'ft2':
        if subsample_mixture is not None:
            assert(abbr_train_file=='all')
            output_dirname += \
                '_mix='+','.join(f'{k}:{v}' for k,v in subsample_mixture.items())
            
    if any(job_name == y for y in ['oi2']):
        output_dirname += f'_ep={num_train_epochs}'
            
    if job_name == 'oi3':
        output_dirname += '_'+mix_name
        
#     if job_name.startswith('oi4'):
    if subsample_inds_file:
        subsample_inds_file_abbr = subsample_inds_file_abbr_fn(subsample_inds_file)
        if subsample_inds_file_abbr:
            output_dirname += f'_{subsample_inds_file_abbr}'

            
    if test_run:
        output_dirname = 'jpt_'+output_dirname
            
    # if not test_run:
    #     output_dirname += \
    #         '_ep='+str(num_train_epochs)
    if add_hardwarespec_to_dirname:
        output_dirname += \
            ('_fsdp='+fsdp.split(' ')[0] if fsdp else '')+\
            ('_deepspeed='+os.path.basename(deepspeed).split('.')[0] if deepspeed else '')+\
            ('_gradckpt='+str(gradient_checkpointing) if gradient_checkpointing else '')+\
            '_mbsz='+str(per_device_train_batch_size)+\
            '_dtype='+torch_dtype+\
            ('_mp='+str(mixed_precision) if mixed_precision else '_mp=none')+\
            '_seqlen='+str(max_seq_length)+\
            '_nodes='+str(nodes)
    if suffix:
        output_dirname += suffix
    output_dir = os.path.join('results', job_name, output_dirname)
    os.makedirs(os.path.join('results', job_name), exist_ok=True)
    wandb_run_name = output_dir.replace('results/', '')
    

    cmd = f"""
    {'!cd .. && ' if test_run else ''}{exe}
        open_instruct/finetune_trainer.py \
        --model_name_or_path={model_name_or_path} \
        --tokenizer_name={model_name_or_path} \
        {'--load_in_8bit' if load_in_8bit else ''} \
        --use_fast_tokenizer=True \
        --train_file={train_file} \
        --max_seq_length={max_seq_length} \
        {'--max_train_samples='+str(max_train_samples) if max_train_samples else ''} \
        {'--use_lora' if use_lora else ''}
        {'--lora_rank='+str(lora_rank) if use_lora else ''}
        {'--lora_alpha='+str(lora_alpha) if use_lora else ''}
        {'--lora_dropout='+str(lora_dropout) if use_lora else ''}
        --do_train \
        --preprocessing_num_workers={preprocessing_num_workers} \
        --per_device_train_batch_size={per_device_train_batch_size} \
        --gradient_accumulation_steps={gradient_accumulation_steps} \
        --learning_rate=2e-5 \
        --lr_scheduler_type={lr_scheduler_type} \
        --warmup_ratio={warmup_ratio} \
        --weight_decay=0. \
        --optim={optimizer} \
        --evaluation_strategy={evaluation_strategy} \
        {'--eval_steps='+str(eval_steps) if eval_steps else ''} \
        {'--report_to '+str(report_to) if report_to else ''} \
        --run_name {wandb_run_name} \
        --logging_strategy=steps \
        --logging_first_step \
        --logging_steps=1 \
        --save_strategy={save_strategy} \
        --save_steps={save_steps} \
        --save_total_limit={save_total_limit} \
        --num_train_epochs={num_train_epochs} \
        --ddp_timeout=7200 \
        {'--fsdp="'+fsdp+'"' if fsdp else ''}
        {'--fsdp_transformer_layer_cls_to_wrap="'+fsdp_transformer_layer_cls_to_wrap+'"' 
            if fsdp else ''}
        {'--gradient_checkpointing' if gradient_checkpointing  else ''}
        --torch_dtype={torch_dtype} \
        --dataloader_num_workers=8 \
        {f'--{mixed_precision}=True' if mixed_precision else ''} \
        {'--overwrite_output_dir' if overwrite_output_dir else ''} \
        {'--deepspeed='+deepspeed if deepspeed else ''} \
        {'--subsample_mixture="'+str(subsample_mixture).replace(': ', ':').replace(', ', ',')+'"'
            if subsample_mixture else ''} \
        {'--subsample_inds_file='+subsample_inds_file if subsample_inds_file else ''} \
        {'--dataloader_sampler '+str(dataloader_sampler) if dataloader_sampler else ''} \
        --use_flash_attn False \
        --low_cpu_mem_usage \
        --overwrite_cache \
        --output_dir="{output_dir}" \
    """ 
    #    --overwrite_cache # if delete a dataset and need to refresh cache

    cmd = multiline_to_singleline(cmd)
    if test_run:
        print()
        print(cmd)

    shell_scripts = shell_scripts_template.format(
        conda_env='open-instruct',
        cwd=os.path.dirname(os.getcwd()),
        cmd=cmd,
        log_dir=os.getcwd(),
        save_dir=output_dir
    )
    out = submit_job(
        shell_scripts, 
        job_name=job_name, 
        nodes=nodes,
        num_cpus=num_cpus,
        cpu_mem=cpu_mem,
        num_gpus=num_gpus,
        gpu_type=gpu_type,
        test_run=test_run,
        job_duration=job_duration,
    )
    if not test_run:
        print(out)